<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/ML/recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://a.teleboss.ru/play/c851b57a-ccd5-408c-9484-e3d00c79b46e

https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system/data?select=movies.csv

# Загрузка датасета с kaggle

In [22]:
!curl -L -o movie-recommendation-system.zip https://www.kaggle.com/api/v1/datasets/download/parasharmanas/movie-recommendation-system
!unzip -o /content/movie-recommendation-system.zip
!rm *.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  164M  100  164M    0     0  39.3M      0  0:00:04  0:00:04 --:--:-- 52.1M
Archive:  /content/movie-recommendation-system.zip
  inflating: movies.csv              
  inflating: ratings.csv             


In [37]:
import numpy as np
import pandas as pd
import os
import warnings
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

#warnings.filterwarnings("ignore")
pd.options.display.max_columns = 100

In [2]:
ratings_full_df = pd.read_csv("/content/ratings.csv", nrows=200_000)
movies_df = pd.read_csv("/content/movies.csv")

In [25]:
ratings_full_df.head(3)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828


In [26]:
movies_df.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [3]:
X_ = ratings_full_df.drop("rating", axis=1)
y_ = ratings_full_df['rating']

# Подготовка pipline для обработки
## Часть 1 - Пишем cамый примитивный Estimator

Он будет использоваться как baseline для дальнейших экспериментов

In [22]:
class NaivePredictor(ClassifierMixin, BaseEstimator):

    def __init__(self, a=0.5, avg=np.median):
        self.rating_movie_mean_ = None  # Для хранения средней оценки фильма
        self.rating_user_mean_ = None  # Для хранения средней оценки которую ставит пользователь
        self.a = a  # Параметр, который будем менять при подборе по сетке GridSearchCV для определения долей влияния средней оценки фильма и среднй оценки пользователя
        self.avg = avg # Параметр, который будем менять при подборе по сетке GridSearchCV для выбора способа оценки среднего выборки
        self.classes_ = [0] # Технический момент, никак не используется, но атрибут должен существовать для корректной работы GridSearchCV

    def fit(self, X, y):
        concatenated_df = pd.concat([X, y], axis=1)
        self.rating_movie_mean_ = concatenated_df.groupby('movieId')['rating'].apply(self.avg) # Средняя оценка фильма
        self.rating_user_mean_ = concatenated_df.groupby('userId')['rating'].apply(self.avg) # Средняя оценка, которую ставит пользователь
        self.global_avg = y.pipe(self.avg) # Средняя всех оценок в тренировочном датасете

        return self

    def predict(self, X):
        """
        Предсказывает средний рейтинг фильма как сумму долей средней оценки фильма и среднй оценки, которую обычно ставит пользователь
        Если `movieId` нет в обучающей выборке, возвращает средний рейтинг по всем фильмам.
        """
        if self.rating_movie_mean_ is None:
            raise ValueError("Model is not fitted yet. Call `fit` before `predict`.")

        predictions = (X['movieId'].map(self.rating_movie_mean_) * self.a +
                       X['userId'].map(self.rating_user_mean_) * (1 - self.a))\
                       .fillna(self.global_avg) # Заполняем пропущенные значения средним рейтингом всех фильмов

        return predictions

In [25]:
# Базовый пайплайн, без трансформеров датасета
pipe = Pipeline(
    [
        ("NaivePredictor", NaivePredictor())
    ]
)

In [15]:
# Для кросс валидации будем использовать KFold
splitter = KFold(
    n_splits=2, # Разбиваем на 5 частей и поочередно используем каждую часть как test
    shuffle=True,
    random_state=42
)

In [24]:
# Для подбора по сетке будем пробовать разные коэффициенты a и разные оценки среднего
param_grid = {
    "NaivePredictor__a": np.arange(start=0.1, stop=1.1, step=0.1),
    "NaivePredictor__avg": [np.mean, np.median]
}

In [26]:
# Перебираем все комбинации из переметров из param_grid
%%time
search_naive = GridSearchCV(pipe,
                      param_grid,
                      cv=splitter,
                      scoring='neg_mean_absolute_error', # В качестве метрики для выбора лучшей модели возьмем MAE
                      verbose=1,
                      return_train_score=True,
                      error_score="raise")

search_naive.fit(X_, y_)
print(f"Best parameter (CV score={search_naive.best_score_:.5f}):")
print(search_naive.best_params_)

Fitting 2 folds for each of 20 candidates, totalling 40 fits
Best parameter (CV score=-0.70413):
{'NaivePredictor__a': 0.4, 'NaivePredictor__avg': <function median at 0x7ed492d3cf30>}
CPU times: user 38.1 s, sys: 181 ms, total: 38.2 s
Wall time: 43.3 s


Best parameter (CV score=-0.69944):
{'NaivePredictor__a': 0.4, 'NaivePredictor__avg': <function median at 0x7bffe353caf0>}

In [4]:
def evaluate_model(y_true, y_predicted):
  '''
  Функция для расчета прочих оценочных метрик модели
  '''
  result = pd.DataFrame.from_dict({
          "MAE": f'{mean_absolute_error(y_true, y_predicted):.3f}',
          "MSE": f'{mean_squared_error(y_true, y_predicted):.3f}',
          "MAPE": f'{mean_absolute_percentage_error(y_true, y_predicted):.2%}',
          "R2_score": f'{r2_score(y_true, y_predicted):.3f}'
  },
  orient='index', columns=['value'])
  return result

In [5]:
# Воспользуемся базовым train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_,
    y_,
    test_size=0.2, # Доля теста к тотал данным
    random_state=42  # Мешать ли данные или делить по индексам
)

In [27]:
# Посмотрим оценки навной модели модели (они довольно хорошие)
evaluate_model(y_test, search_naive.predict(X_test))

,value
MAE,0.666
MSE,0.792
MAPE,30.93%
R2_score,0.273


In [28]:
# Заранее зафитим MultiLabelBinarizer, который будет преобразовывать колонку с жанрами в отельные столбцы
mlb_genres = MultiLabelBinarizer()
mlb_genres.fit(movies_df.genres.apply(lambda x: x.split('|')))

MultiLabelBinarizer()

In [32]:
# Напишем свой класс для предварительной трансформации данных
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        movies_data=movies_df, # база данных всех фильмов
        avg=np.mean, # для дальнейшей проверки в GridSearchCV, что будет лучше работать mran или median

    ):

        self.mlb = mlb_genres
        self.movies_data = movies_data
        self.avg = avg

    def fit(self, X, y):


        X_train_copy = X.copy()
        X_train_copy = pd.concat([X_train_copy, y], axis = 1)
        X_train_copy = X_train_copy.merge(movies_df[['movieId', 'genres']], on='movieId', how='left')

        #  Для каждого user в train рассчитаем среднюю оценку, которую он ставит фильмам
        self.rating_user_mean = X_train_copy.groupby("userId")\
                                               .rating.apply(self.avg)\
                                               .rename("rating_user_mean")

        #  Для каждого movie в train рассчитаем среднюю оценку, которую он получает от пользователь
        self.rating_movie_mean = X_train_copy.groupby("movieId")\
                                                .rating.apply(self.avg)\
                                                .rename("rating_movie_mean")

        # Для каждого пользователя рассчитаем среднюю оценку, которую он ставит в каждом жанре фильма, для чего сначала выдлим все жанры и проставим им оценки для каждой записи train
        mlb_encoded_genres = pd.concat([
            X_train_copy.userId,
            pd.DataFrame(self.mlb.transform(X_train_copy.genres.apply(lambda x: x.split('|'))), \
                                          columns=self.mlb.classes_, index = X_train_copy.index).mul(X_train_copy.rating, axis=0)
        ], axis=1)
        # А затем сгуппируем по пользователям и усредним
        self.mean_genres_by_users = mlb_encoded_genres.groupby("userId").apply(self.avg, include_groups=False).rename("mean_genres_by_users")
        self.mean_rating = y.pipe(self.avg) # средний рейтинг всех фильмов train для заполнения пропусков в дальнейшем при трансформации X_test
        return self

    def transform(self, X:pd.DataFrame):

        X_copy= X.copy()
        # Добавим к X_test иформацию их X_train о средних для фильма и средних для пользователя рейтингах. Пропуски заполним средним рейтингом всех фильмов.
        X_copy = X_copy.merge(self.movies_data[['movieId', 'genres']], on='movieId', how='left')\
                       .merge(self.rating_user_mean, left_on='userId', right_index=True, how='left')\
                       .merge(self.rating_movie_mean, left_on='movieId', right_index=True, how='left').fillna(self.mean_rating)

        datetime_ = pd.to_datetime(X_copy.timestamp, unit='s')
        """
        # Эти признаки в дальнейшем показали свою бесполезность
        X_copy['year'] = datetime_.dt.year
        X_copy['day'] = datetime_.dt.day
        X_copy['hour'] = datetime_.dt.hour
        X_copy['month'] = datetime_.dt.month
        """
        X_copy['day_of_week'] = datetime_.dt.day_of_week

        # Создадим признак того, в какое время дня был посмотрен фильм. В дальнейшем применим на эту колонку OneHotEncoder
        X_copy['time'] = pd.cut(datetime_.dt.hour,
                                include_lowest=True,
                                bins=[0, 5, 12, 17, 21, 24],
                                right=False,
                                ordered=False,
                                labels=['night', 'morining', 'afternoon', 'evening', 'night'])


        # добавим к X_train колонки, кодирующие признак, к какому жанру принадлежит текущий фильм
        X_copy = pd.concat([X_copy, pd.DataFrame(self.mlb.transform( X_copy.genres.apply(lambda x: x.split('|'))),\
                                         columns=self.mlb.classes_)], axis=1)
        # добавим к X_train колонки с информацией о том, какие в среднем оценки получает каждый жанр от текущего пользователя
        X_copy = X_copy.merge(self.mean_genres_by_users, how='left', left_on='userId', right_index=True, suffixes=("", "_mean"))
        # дропнем кололнки, которые мы используем трансформированными и те, что нам точно не пригодятся для предсказания рейтинга фильма
        X_copy.drop(columns=['timestamp', 'genres', 'userId',  'movieId',], inplace=True)
        return X_copy.set_index(X.index) # для надежности вернем первоначальный индекс, в X_test


In [9]:
onehot_columns = ['time'] # список колонок, которые будем кодировать c OneHotEncoder

In [34]:
col_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore'), onehot_columns),
    ],
    remainder='passthrough',          # Оставляем необрабатываемые колонки как есть, не удаляем их
    verbose_feature_names_out=False   # Оставляем оригинальные названия колонок
).set_output(transform='pandas')      # Трансформер будет возвращать pandas


In [38]:
# Промежуточный pipeline для иссдледования Variance Inflation Factor
research_pipe = Pipeline(
    [
        ("Custom_transformer", CustomTransformer()),
        ("OneHotEncoder", col_transformer)
    ]
)

In [39]:
df = research_pipe.fit(X_, y_).transform(X_)

In [40]:
vif_df = pd.DataFrame()
vif_df['feature'] = df.columns
vif_df['VIF'] = [vif(df.values, i) for i in range(len(df.columns))]
vif_df.query("VIF >= 10")

,feature,VIF
3,rating_user_mean,136.420670
4,rating_movie_mean,39.907734
26,mean_genres_by_users,90.423098


Некоторые признаки, такие как rating_user_mean, mean_genres_by_users, rating_movie_mean имеют достаточно высоку коллинеарность, что логично, так как методы их получения в чем-то схожи, особенно для первых двух. Однако их удаление из датасета ведет к ухудшению показателей точности модели, поэтому пока трогать их не будем

In [11]:
lasso_pipe = Pipeline(
    [
        ("Custom_transformer", CustomTransformer()),
        ("OneHotEncoder", col_transformer),
        ("simple_model", Lasso())
    ]
)

ridge_pipe = Pipeline(
    [
        ("Custom_transformer", CustomTransformer()),
        ("OneHotEncoder", col_transformer),
        ("simple_model", Ridge())
    ]
)

In [13]:
param_grid = {
    "simple_model__alpha": np.logspace(-1, 3, 10),
    "simple_model__max_iter": [100, 1000],
    "Custom_transformer__avg": [np.mean, np.median],
}

In [16]:
%%time
for model in [lasso_pipe, ridge_pipe]:

    search = GridSearchCV(model,
                          param_grid,
                          cv=splitter,
                          scoring='neg_mean_absolute_error', # В качестве метрики для выбора лучшей модели возьмем MAE
                          verbose=1,
                          return_train_score=True,
                          error_score="raise")

    search.fit(X_, y_)

    print(f"Best parameter (CV score={search.best_score_:.5f}):")
    print(search.best_params_)

Fitting 2 folds for each of 40 candidates, totalling 80 fits
Best parameter (CV score=-0.71554):
{'Custom_transformer__avg': <function mean at 0x7ed49603f0f0>, 'simple_model__alpha': 0.1, 'simple_model__max_iter': 100}
Fitting 2 folds for each of 40 candidates, totalling 80 fits
Best parameter (CV score=-0.69073):
{'Custom_transformer__avg': <function mean at 0x7ed49603f0f0>, 'simple_model__alpha': 1000.0, 'simple_model__max_iter': 100}
CPU times: user 8min 20s, sys: 51.1 s, total: 9min 11s
Wall time: 9min 6s


In [17]:
search.best_estimator_.named_steps.simple_model.coef_

array([ 5.48430155e-03, -3.89592394e-03,  4.21171076e-02,  8.06560429e-01,
        8.38133612e-01,  4.10759130e-03,  9.96480370e-03, -1.43076372e-02,
       -1.11854837e-02,  6.24980239e-03, -3.56794509e-02, -1.34233272e-02,
        9.04149766e-03,  5.37109965e-02,  1.14718236e-02, -2.17514960e-03,
       -8.34547114e-04,  2.18842782e-02, -5.89496854e-02,  7.17227428e-03,
        1.38765545e-03, -1.34405302e-02, -2.65254672e-02, -1.10223158e-03,
        5.06370952e-04,  2.11589646e-02,  1.34247725e-01])

In [18]:
evaluate_model(y_test, search.predict(X_test))

,value
MAE,0.636
MSE,0.689
MAPE,27.22%
R2_score,0.367
